In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights

In [2]:
model = models.vit_b_16(weights=None)

In [3]:
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
import os
import torchvision.transforms as transforms
from collections import OrderedDict

In [4]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 10))]))
model = model.to(device)

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [5]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[-0.6266,  0.4368, -0.7189,  0.0673, -0.9845,  0.0078, -0.5048,  0.3943,
         -0.4504,  0.0396]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [6]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'untrained_torchvit'), None)

Epoch 10 done, train loss: 0.0288 val acc: 0.8170
Epoch 20 done, train loss: 0.0166 val acc: 0.8800
Epoch 30 done, train loss: 0.0167 val acc: 0.8340
Epoch 40 done, train loss: 0.0265 val acc: 0.8835
Epoch 50 done, train loss: 0.0027 val acc: 0.9095
Epoch 60 done, train loss: 0.0057 val acc: 0.9165
Epoch 70 done, train loss: 0.0101 val acc: 0.9040
Epoch 80 done, train loss: 0.0001 val acc: 0.9470
Epoch 90 done, train loss: 0.0000 val acc: 0.9470
Epoch 100 done, train loss: 0.0000 val acc: 0.9475


In [7]:
model = torch.load(os.path.join('..', 'models', 'untrained_torchvit', 'digit-model.pt'))

In [ ]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

In [ ]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)